In [7]:
import os
import pcse
import yaml
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
from pcse.db import NASAPowerWeatherDataProvider
from pcse import fileinput as fi

In [8]:
pts = gpd.read_file('../../CIAT/data/BEM_2019_Chiapas_Final_Clean.gpkg')
pts['site'] = ['CH{0:05d}'.format(i + 1) for i in range(len(pts))]
pts['lon'] = pts.centroid.x
pts['lat'] = pts.centroid.y
pts['ref_date'] = pd.to_datetime(pts.fecha_de_siembra)
pts['nrate'] = pts.nitrogen/10


In [9]:
soilpts = gpd.read_file('../data/CH_Soils.gpkg')
soilpts.geometry = soilpts.to_crs('epsg:32715').buffer(5000).to_crs(pts.crs)
pts = gpd.sjoin(pts, soilpts, how = 'left')

/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [10]:
for i in trange(len(pts)):
    wdp = NASAPowerWeatherDataProvider(latitude=pts.lat.values[i], longitude=pts.lon.values[i])

    save_cols = ['DAY', 'IRRAD', 'TMIN', 'TMAX', 'VAP', 'WIND', 'RAIN']
    wdpdf = pd.DataFrame(wdp.export()).loc[:,save_cols]
    wdpdf['DAY'] = pd.to_datetime(wdpdf.DAY).dt.strftime('%Y%m%d')
    wdpdf['IRRAD'] = wdpdf.IRRAD / 1000
    wdpdf['VAP'] = wdpdf.VAP / 10
    wdpdf['RAIN'] = wdpdf.RAIN * 10
    wdpdf['SNOWDEPTH'] = np.nan


    site_c = ["## Site Characteristics",
              "Country     = 'Mexico'",
              "Station     = '{}'".format(pts.site.values[i]),
              "Description = 'MASAGRO'",
              "Source      = 'Daymet'",
              "Contact     = 'Rodrigo'",
              "Longitude   = {}".format(wdp.longitude),
              "Latitude    = {}".format(wdp.latitude),
              "Elevation   = {}".format(wdp.elevation),
              "AngstromA   = {}".format(wdp.angstA),
              "AngstromB   = {}".format(wdp.angstB),
              "HasSunshine = False",
              "## Daily weather observations",
              ','.join(wdpdf.columns)
             ]

    wfile_np = '/home/rodrigo7/Apsim_test/MASAGRO/PCSE/{}_np.csv'.format(pts.site.values[i])
    with open(wfile_np, 'w') as f:
        for r in site_c:
            f.writelines(r)
            f.writelines('\n')
    wdpdf.to_csv(wfile_np, na_rep = 'NaN', mode = 'a', float_format = '%.3f', header = False, index = False)

In [11]:
wdp = fi.CSVWeatherDataProvider(wfile_np)


In [12]:
print(wdp)

Weather data provided by: CSVWeatherDataProvider
--------Description---------
Weather data for:
Country: Mexico
Station: CH04594
Description: MASAGRO
Source: Daymet
Contact: Rodrigo
----Site characteristics----
Elevation:  768.5
Latitude:  17.200
Longitude: -92.700
Data available for 1983-07-01 - 2020-02-28
Number of missing days: 23

